In [13]:
import numpy as np
from scipy.spatial import distance
import pandas as pd
from scipy import stats
import operator
from sklearn.model_selection import KFold
from statistics import mean
from datetime import datetime

np.set_printoptions(suppress=True) #prevent numpy exponential

class Tree_node(object):
    
    question_val = None
    true_child = None
    false_child = None
    feature_ind = None
    label = None
    
    def __init__(self, question_val, true_child, false_child, feature_ind, label):
        self.question_val = question_val
        self.true_child = true_child
        self.false_child = false_child
        self.feature_ind = feature_ind
        self.label = label


def read_data(file):
    nominal = dict()
    gene_data = open(file)
    gene_seq = gene_data.readlines()
    all_genes_list = []
    first_row = gene_seq[0].split("\t")
    for f in range(len(first_row)):
        try:
            float(first_row[f])
        except:
            nominal[f] = []
    for line in gene_seq:
        gene = line.strip().split("\t")
        for f in range(len(gene)):
            try:
                gene[f] = float(gene[f])
            except:
                category = nominal[f]
                if gene[f] in category:
                    gene[f] = float(category.index(gene[f]))
                else:
                    category.append(gene[f])
                    gene[f] = float(category.index(gene[f]))
        all_genes_list.append(gene)
    return nominal, np.asarray(all_genes_list, dtype = float)

def compute_giniIndex(feature_col, train_classes, split_val,is_nominal):
    true_dict = dict()
    true_cnt = 0
    false_dict = dict()
    false_cnt = 0
    for row in range(len(feature_col)):
        
        if((is_nominal and (feature_col[row] == split_val)) or (not(is_nominal) and (feature_col[row] < split_val))):
            # increment true dict count for corresponding class
            if train_classes[row] in true_dict:
                true_dict[train_classes[row]] += 1
            else:
                true_dict[train_classes[row]] = 1
            true_cnt +=1
        else:
            # increment false dict count for corresponding class
            if train_classes[row] in false_dict:
                false_dict[train_classes[row]] += 1
            else:
                false_dict[train_classes[row]] = 1
            false_cnt += 1
                
    if(len(true_dict) == 0):
        true_gini = 1.0
    else:
        proportion = 0.0
        for c in true_dict:
            p = (true_dict[c]/true_cnt)
            proportion += p**2
        true_gini = 1-proportion
        
    if(len(false_dict) == 0):
        false_gini = 1.0
    else:
        proportion = 0.0
        for c in false_dict:
            p = (false_dict[c]/false_cnt)
            proportion += p**2
        false_gini = 1-proportion
    # final gini from all groups
    return ((true_cnt/len(train_classes))*true_gini) + ((false_cnt/len(train_classes))*false_gini)
    

def find_best_split(train_data,train_classes,nominal_dict):
    final_gini = 1
    final_question_val = None
    final_feature_ind = 0
    for f in range(len(train_data[0])):  #iterate features
        is_nominal = False 
        computed_ginis = set()
        local_gini = 1
        local_ques = None
        local_feature_ind = 0
        if f in nominal_dict:
            is_nominal = True
        for r in range(len(train_data)):  # iterate every record for the feature
            if train_data[r][f] in computed_ginis:
                continue
            gini = compute_giniIndex(train_data[:,f], train_classes, train_data[r][f],is_nominal)
            if gini < local_gini:
                local_gini = gini
                local_ques = train_data[r][f]
                local_feature_ind = f
            computed_ginis.add(train_data[r][f])
        
        if local_gini < final_gini:
                final_gini = local_gini
                final_question_val = local_ques
                final_feature_ind = local_feature_ind
#     print("final gini: ",final_gini)
#     print('ques: ',final_question_val)
#     print("features_ind: ",final_feature_ind)
    return final_question_val,final_feature_ind
    
def assign_majority_class(labels):
    print("in majority compute")
    unique_labels = np.unique(labels)
    max_cnt = 0
    majority_class = None
    for c in unique_labels:
        count = np.count_nonzero(labels == c)
        if count > max_cnt:
            max_cnt = count
            majority_class = c
    return majority_class
    
def compute_tree(train_data,train_classes,nominal_dict):
    if(len(np.unique(train_classes)) <= 1):
        #print("leaf node label: ",int(train_classes[0]))
        return Tree_node(None,None,None,None,train_classes[0])
    nominal = False
    question_val, feature_ind = find_best_split(train_data,train_classes,nominal_dict)
#     print("Node data: ")
#     print("Split column Ind: ", int(feature_ind))
#     print("Split criteria: ",nominal_dict[feature_ind][int(question_val)])
    if feature_ind in nominal_dict:
        nominal = True
    cur_node = Tree_node(question_val,None,None,feature_ind,None)
    true_data = []
    true_classes = []
    false_data = []
    false_classes = []
    for row in range(len(train_data[:,feature_ind])):
        if((nominal and (train_data[row][feature_ind] == question_val)) or (not(nominal) and (train_data[row][feature_ind] < question_val))):
            #Populating left-->true side data
            true_data.append(train_data[row])
            true_classes.append(train_classes[row])
        else:
            #Populating right-->false side data
            false_data.append(train_data[row])
            false_classes.append(train_classes[row])

    #Case when all the input rows are classied to one of the child nodes.
    if len(true_data)==0:
        return Tree_node(None,None,None,None,assign_majority_class(false_classes))
    if len(false_data)==0:
        return Tree_node(None,None,None,None,assign_majority_class(true_classes))
    
    #print("Computing true(left) child node")
    cur_node.true_child = compute_tree(np.asarray(true_data),true_classes,nominal_dict)
    
    cur_node.false_child = compute_tree(np.asarray(false_data),false_classes,nominal_dict)
    #print("Computing false(Right) child node")
    
    return cur_node

def get_class_from_tree(test_data, root, nominal_dict):
    if root.label != None:
        return root.label
    col = root.feature_ind
    val = root.question_val
    nominal = False
    if col in nominal_dict:
        nominal = True
    if((nominal and (test_data[col] == val)) or (not(nominal) and (test_data[col] < val))):
        return get_class_from_tree(test_data, root.true_child, nominal_dict)
    else:
        return get_class_from_tree(test_data, root.false_child, nominal_dict)

def predict_test_classes(test_data, root, nominal_dict):
    predicted_classes = []
    for i in range(len(test_data)):
        test_class = get_class_from_tree(test_data[i], root, nominal_dict)
        predicted_classes.append(test_class)
    return predicted_classes
    
def calculate_metrics(predicted_classes, ground_truth):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i in range(0, len(predicted_classes)):
        if(predicted_classes[i] == 1 and ground_truth[i] == 1):
            tp += 1
        elif(predicted_classes[i] == 1 and ground_truth[i] == 0):
            fp += 1
        elif(predicted_classes[i] == 0 and ground_truth[i] == 1):
            fn += 1
        else:
            tn += 1
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    if (tp+fp) != 0:
        precision = tp / (tp + fp)
    else:
        precision = 0
    if (tp+fn) != 0:
        recall = tp / (tp + fn) 
    else:
        recall = 0
    if ((2 * tp) + fp + fn) != 0:
        f_1_measure = (2 * tp) / ((2 * tp) + fp + fn) 
    else:
        f_1_measure = 0
    return accuracy, precision, recall, f_1_measure

def print_decision_tree(root,nominal_dict):
    if(root.label != None):
        print("Leaf Node with label: ",int(root.label))
        return
    if(root.true_child != None):
        if(root.true_child.label == None):
            print("Left Child: {",int(root.true_child.feature_ind),", ",nominal_dict[root.true_child.feature_ind][int(root.true_child.question_val)],"}")
    if(root.false_child != None):
        if(root.false_child.label == None):
            print("Right Child: {",int(root.false_child.feature_ind),", ",nominal_dict[root.false_child.feature_ind][int(root.false_child.question_val)],"}")
    if(root.true_child != None):
        print_decision_tree(root.true_child,nominal_dict)
    if(root.false_child != None):
        print_decision_tree(root.false_child,nominal_dict)

nominal_dict, data = read_data("project3_dataset4.txt")
classes = data[:,len(data[0])-1]
feature_data = data[:,:len(data[0])-1]
train_data = feature_data
train_classes = classes
#print(len(train_classes))
root_node = compute_tree(train_data,train_classes,nominal_dict) 
# print(root_node.feature_ind)
# print(nominal_dict[root_node.feature_ind][int(root_node.question_val)])
print("Root Node split-criteria: {",int(root_node.feature_ind),",",nominal_dict[root_node.feature_ind][int(root_node.question_val)],"}")
print_decision_tree(root_node,nominal_dict)


Root Node split-criteria: { 0 , overcast }
Right Child: { 2 ,  high }
Leaf Node with label:  1
Left Child: { 0 ,  sunny }
Right Child: { 3 ,  weak }
Right Child: { 3 ,  weak }
Leaf Node with label:  0
Leaf Node with label:  1
Leaf Node with label:  0
Right Child: { 0 ,  rain }
Leaf Node with label:  1
Leaf Node with label:  0
Leaf Node with label:  1
